# MLOps Zoomcamp
## Homework 1

Lucas Augusto Müller de Souza

Data Engineer at Cinnecta (lucas.muller@cinnecta.com)


www.linkedin.com/in/lucas-mullers

In [1]:
!wget -nc 'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet'
!wget -nc 'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet'

File ‘fhv_tripdata_2021-01.parquet’ already there; not retrieving.

File ‘fhv_tripdata_2021-02.parquet’ already there; not retrieving.



In [2]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
def read_data(filename=''):
    return pd.read_parquet(filename)

def data_prep(df, dv):
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df['duration'].dt.total_seconds() / 60.0
    
    df = df[(df['duration'] >= 1.0) & (df['duration'] <= 60)]
    
    values = {"PUlocationID": -1, "DOlocationID": -1}
    df = df.fillna(value=values)
    
    X_df = df[['PUlocationID', 'DOlocationID']].astype(str)
    df_dicts = X_df.to_dict(orient='records')
    X_df = dv.transform(df_dicts)
    
    y_df = df['duration'].values
    
    return X_df, y_df

# Q1. Downloading the data


We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records"

Download the data for January and February 2021

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

- 1054112
- 1154112 [X]
- 1254112
- 1354112

In [4]:
jan_df = pd.read_parquet('fhv_tripdata_2021-01.parquet')
jan_df.shape

(1154112, 7)

# Q2. Computing duration


Now let's compute the duration variable. It should contain the duration of a ride in minutes.


What's the average trip duration in January?


- 15.16
- 19.16 [X]
- 24.16
- 29.16

In [5]:
jan_df['duration'] = jan_df.dropOff_datetime - jan_df.pickup_datetime
jan_df['duration'] = jan_df['duration'].dt.total_seconds() / 60.0

In [6]:
jan_df['duration'].mean()

19.1672240937939

# Data preparation

Check the distribution of the duration variable. There are some outliners.

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

How many records did you drop?

In [7]:
jan_df[(jan_df['duration'] >= 1.0) & (jan_df['duration'] <= 60)].shape[0]

1109826

## Difference

In [8]:
jan_df.shape[0] - jan_df[(jan_df['duration'] >= 1.0) & (jan_df['duration'] <= 60)].shape[0]

44286

In [9]:
filtered_df = jan_df[(jan_df['duration'] >= 1.0) & (jan_df['duration'] <= 60)]

# Q3. Missing values

The features we'll use for our model are the pickup and dropoff location IDs.

But they have a lot of missing values there. Let's replace them with "-1"

What's the factions of missing values for the pickup location ID? (Or the fraction of "-1"s after you filled the NAs)

- 53%
- 63%
- 73%
- 83% [X]

In [10]:
filtered_df.describe()

,PUlocationID,DOlocationID,duration
count,182818.000000,961919.000000,1.109826e+06
mean,139.861633,135.689147,1.624725e+01
std,74.763788,80.349678,1.155150e+01
min,1.000000,1.000000,1.000000e+00
25%,75.000000,67.000000,7.850000e+00
50%,146.000000,132.000000,1.323333e+01
75%,206.000000,213.000000,2.146667e+01
max,265.000000,265.000000,6.000000e+01


In [11]:
filtered_df.isna().sum() / filtered_df.shape[0] * 100

dispatching_base_num        0.000000
pickup_datetime             0.000000
dropOff_datetime            0.000000
PUlocationID               83.527328
DOlocationID               13.327044
SR_Flag                   100.000000
Affiliated_base_number      0.069651
duration                    0.000000
dtype: float64

In [12]:
values = {"PUlocationID": -1, "DOlocationID": -1}
filtered_df = filtered_df.fillna(value=values)

# Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries

Fit a dictionary vectorizer

Get a feature matrix from it

What's the dimensionality of this matrix? (The number of columns)

- 2
- 152
- 352
- 525 [X]
- 725

In [13]:
X_train = filtered_df[['PUlocationID', 'DOlocationID']].astype(str)
X_train

,PUlocationID,DOlocationID
0,-1.0,-1.0
1,-1.0,-1.0
3,-1.0,72.0
4,-1.0,61.0
5,-1.0,71.0
...,...,...
1154107,7.0,7.0
1154108,44.0,91.0
1154109,171.0,171.0
1154110,15.0,15.0


In [14]:
y_train = filtered_df['duration'].values
y_train

array([17.        , 17.        ,  8.28333333, ..., 16.2       ,
       19.43333333, 36.        ])

In [15]:
train_dicts = X_train.to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

# Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

- 5.52
- 10.52 [X]
- 15.52
- 20.52

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_predicted = lr.predict(X_train) 

rmse = mean_squared_error(y_train, y_predicted, squared=False)
rmse

10.528519107212679

# Q6. Evaluating the model
Now let's apply this model to the validation dataset (Feb 2021).

What's the RMSE on validation?

- 6.01
- 11.01
- 16.01
- 21.01


In [17]:
df_test = read_data(filename='fhv_tripdata_2021-02.parquet')
X_test, y_test = data_prep(df_test, dv)

In [18]:
y_predicted = lr.predict(X_test) 

rmse = mean_squared_error(y_test, y_predicted, squared=False)
rmse

11.014283199431478